In [10]:
from bs4 import BeautifulSoup     #网页解析，获取数据
import re       #正则表达式，进行文字匹配
import urllib.request,urllib.error      #制定URL，获取网页数据
import xlwt     #进行excel操作
import sqlite3  #进行SQLite数据库操作
import pandas as pd #存储数据及处理数据
import numpy as np
import requests
import random
import time
from collections import defaultdict
from collections import deque
from collections import Counter
import collections
import json
import sys
import datetime
import os
import math
from decimal import *
import threading


In [2]:
_=float('inf')
def prim(graph,n):
    for i in range(n):
        for j in range(n):
            if graph[i][j] >0:
                graph[i][j] = float(1)/graph[i][j]
            if graph[i][j] == 0:
                graph[i][j] = _
    dis=[0]*n
    pre=[0]*n
    flag=[False]*n
    flag[0]=True
    k=0
    for i in range(n):
        dis[i]=graph[k][i]
    # print dis
    for j in range(n-1):
        mini=_
        for i in range(n):
            if mini>dis[i] and not flag[i]:
                mini=dis[i]
                k=i
        # print k

        if k==0:
            return
        flag[k]=True
        for i in range(n):
            if dis[i]>graph[k][i] and not flag[i]:
                dis[i]=graph[k][i]
                pre[i]=k
        # print dis
    dis = [float(1)/i for i in dis]
    return dis,pre

In [16]:
# 读取行业小类数据
df_ind_small_11=pd.read_excel('E:\\pingxiang\\11AND17_INDUSTRY.xlsx',sheet_name='11_small')
df_ind_small_17=pd.read_excel('E:\\pingxiang\\11AND17_INDUSTRY.xlsx',sheet_name='17_small')

In [20]:
ls_small_17=df_ind_small_17['GB/T 4754-2017'].tolist()
ls_small_17=list(Counter(ls_small_17))
len(ls_small_17)

1381

In [36]:
ls_small_11=df_ind_small_11['GB/T 4754-2011'].tolist()
ls_small_11=list(Counter(ls_small_11))
len(ls_small_11)

1094

In [39]:
newlinkdf = pd.read_csv('E:\\pingxiang\\addProducerService_weighted23f3_0.25_pp_2022_productService.csv',encoding='gbk')
newlinkdf

,Source,Target,Weight
0,111,5111,0.288136
1,111,5112,0.235955
2,111,5113,0.214286
3,111,5114,0.271186
4,111,5115,0.237288
...,...,...,...
130187,5919,8930,0.015625
130188,5919,8941,0.078125
130189,5919,8942,0.250000
130190,5919,8949,0.203125


In [43]:
newlinkdf = newlinkdf.loc[(newlinkdf['Source'].isin(ls_small_11))&(newlinkdf['Target'].isin(ls_small_11))]
newlinkdf

,Source,Target,Weight
0,111,5111,0.288136
1,111,5112,0.235955
2,111,5113,0.214286
3,111,5114,0.271186
4,111,5115,0.237288
...,...,...,...
130187,5919,8930,0.015625
130188,5919,8941,0.078125
130189,5919,8942,0.250000
130190,5919,8949,0.203125


In [44]:
codedf = pd.concat([newlinkdf[['Source']],newlinkdf[['Target']].rename(columns={'Target':'Source'})],ignore_index=True).drop_duplicates()
codedf = codedf.rename(columns={'Source':'id'}).sort_values('id',ascending=True).reset_index().drop('index',axis=1)
df = pd.DataFrame(index=pd.MultiIndex.from_product([list(codedf['id']),list(codedf['id'])],names=['Source','Target'])).reset_index()
df= pd.merge(df,newlinkdf[['Source','Target','Weight']],left_on=['Source','Target'],right_on=['Source','Target'],how='outer').fillna(0)
df = df.pivot(index='Source',columns='Target',values='Weight')
graph = df.values.tolist()

In [45]:
for i in range(len(graph)):
    for j in range(len(graph)):
        graph[i][j] = max(graph[i][j],graph[j][i])
        graph[j][i] = max(graph[i][j], graph[j][i])
n = len(graph)
dis,pre = prim(graph,n)

In [46]:
mstdf = pd.DataFrame(columns=['Source','Target','Weight'])
mstdf['Source'] = [codedf['id'].values.tolist()[i] for i in pre]
mstdf['Target'] = codedf['id']
mstdf['Weight'] = dis
mstdf = mstdf[mstdf['Source']!=mstdf['Target']]
mstdf = pd.merge(mstdf,codedf,left_on='Source',right_on='id',how='left').drop('id',axis=1)
mstdf = pd.merge(mstdf,codedf,left_on='Target',right_on='id',how='left').drop('id',axis=1)
mstdf['Source'] = [str(i) for i in mstdf['Source']]
mstdf['Target'] = [str(i) for i in mstdf['Target']]
mstdf['Source'][mstdf['Source'].str.len() <3] = mstdf['Source'].apply(lambda x: '0' + str(x))
mstdf['Target'][mstdf['Target'].str.len() <3] = mstdf['Target'].apply(lambda x: '0' + str(x))
mstdf['Type'] = 'undirected'
print('node number is ', codedf.shape[0], 'edge number is ', mstdf.shape[0])
mstdf.to_csv('E:\\pingxiang\\addProducerService_weighted23f3_0.25_pp_2017_productService_MST_0_dys.csv',encoding='gbk',index=False)

node number is  838 edge number is  837


<ipython-input-46-fae348b44599>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mstdf['Source'][mstdf['Source'].str.len() <3] = mstdf['Source'].apply(lambda x: '0' + str(x))
<ipython-input-46-fae348b44599>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mstdf['Target'][mstdf['Target'].str.len() <3] = mstdf['Target'].apply(lambda x: '0' + str(x))


In [47]:
pd_all = pd.read_csv('E:\\pingxiang\\addProducerService_weighted23f3_0.25_pp_2022_productService.csv',encoding='gbk')
pd_mst = pd.read_csv('E:\\pingxiang\\addProducerService_weighted23f3_0.25_pp_2017_productService_MST_0_dys.csv',encoding='gbk')
pd_mst = pd_mst.loc[:,['Source','Target','Weight']]
len_mst = len(pd_mst)
pd_mst['tempsource'] = pd_mst['Source']
pd_mst_ = pd_mst.copy()
pd_mst_['Source'] = pd_mst_['Target']
pd_mst_['Target'] = pd_mst_['tempsource']
pd_mst_  = pd.concat([pd_mst,pd_mst_])
pd_mst_.drop('tempsource',axis = 1,inplace = True)
pd_mst.drop('tempsource',axis = 1,inplace = True)
pd_merge = pd.merge(pd_all,pd_mst_, on=['Source','Target'],how='left')
pd_merge.fillna(-1,inplace=True)
pd_merge = pd_merge[pd_merge['Weight_y']<0]
print (len(pd_merge))
pd_merge.rename(columns={'Weight_x':'Weight'},inplace=True)
pd_merge = pd_merge.loc[:,['Source','Target','Weight']]
pd_merge = pd_merge.sort_values('Weight',ascending=False)
threshold = pd_merge.iloc[len_mst+1,2]
#threshold = 0.78
pd_merge = pd_merge[pd_merge['Weight']>=threshold]
pd_result = pd.concat([pd_merge,pd_mst])
pd_result['Type'] = 'undirected'
print ('node count is ',len_mst,';edge count is ',len(pd_result))
pd_result.to_csv('E:\\pingxiang\\addProducerService_weighted23f3_0.25_pp_2022_productService'+'_'+str(threshold)+'_1.csv',index=False,encoding='utf-8')

129355
node count is  837 ;edge count is  1684
